In [1]:
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
from shapely.geometry import box
from shapely.ops import unary_union
import pandas as pd


In [2]:
# Inputs
# Pick ONE landform layer:
#landform_gdf = gpd.read_file("teton_outline_final.shp")                    # glaciers
landform_gdf = gpd.read_file("Snow_field_threshold_w_lidar_difference.shp")  # snowfields
#landform_gdf = gpd.read_file("grte_updated_RGI_2025_Project.shp")            # rock glaciers

slope_fp = "Teton_slope_2022_reprojected.tif"   # degrees
dhdt_fp  = "local_hyps_1967_2014_elev_rate.tif"                # dh/dt map used for uncertainty. use USGS_2014_2022_lidar_dhdt_reprojected.tif for 2014-2022 uncertainity

In [3]:
# Parameters
L_m = 500.0               # decorrelation length (m) for Rolstad scaling
outer_buffer_m = 1000.0   # stable zone buffer radius
exclude_buffer_m = 10.0   # exclusion ring around landforms to avoid edges
slope_thresh_deg = 30.0   # "stable" pixels have slope < this threshold
min_stable_px = 50        # require at least N stable pixels

print("CRS of shapefile (original):", landform_gdf.crs)

CRS of shapefile (original): COMPD_CS["NAD83(2011) / UTM zone 12N + NAVD88 height",PROJCS["NAD83(2011) / UTM zone 12N",GEOGCS["NAD83(2011)",DATUM["NAD83_National_Spatial_Reference_System_2011",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","1116"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","6318"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-111],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","6341"]],VERT_CS["NAVD88 height",VERT_DATUM["North American Vertical Datum 1988",2005,AUTHORITY["EPSG","5103"]],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Gravity-related height",UP],AUTHORITY["EPSG","5703"]]]


In [4]:
records = []

with rasterio.open(slope_fp) as slope_src, rasterio.open(dhdt_fp) as dhdt_src:
    print("CRS of slope raster:", slope_src.crs)
    print("CRS of dh/dt raster:", dhdt_src.crs)

    # Reproject polygons to dh/dt CRS for buffering and intersections
    if landform_gdf.crs != dhdt_src.crs:
        landform_gdf = landform_gdf.to_crs(dhdt_src.crs)

    # Build the 10 m exclusion union *after* reprojecting to dh/dt CRS
    combined_10m_buffers = unary_union(landform_gdf.buffer(exclude_buffer_m).geometry)

    raster_bounds_polygon = box(*dhdt_src.bounds)

    for idx, row in landform_gdf.iterrows():
        geom = row.geometry
        if geom is None or not geom.is_valid:
            continue
        if not geom.intersects(raster_bounds_polygon):
            continue

        try:
            #Stable zone in dh/dt CRS: 1 km buffer minus 10 m exclusion, clipped to raster bounds
            buf_outer = geom.buffer(outer_buffer_m)
            stable_zone = buf_outer.difference(combined_10m_buffers).intersection(raster_bounds_polygon)
            if stable_zone.is_empty:
                print(f"landform {idx}: empty stable zone")
                continue

            #Mask dh/dt within stable zone (in dh/dt CRS)
            dhdt_img, dhdt_tx = mask(dhdt_src, [stable_zone], crop=True, filled=True, nodata=np.nan)
            dhdt_arr = dhdt_img[0]

            #Mask slope within stable zone: reproject polygon to slope CRS, crop slope there
            stable_zone_slope_crs = gpd.GeoSeries([stable_zone], crs=dhdt_src.crs).to_crs(slope_src.crs).iloc[0]
            slope_img, slope_tx = mask(slope_src, [stable_zone_slope_crs], crop=True, filled=True, nodata=np.nan)

            #Reproject slope subset onto the dh/dt subset grid so shapes match
            slope_on_dhdt = np.full(dhdt_arr.shape, np.nan, dtype=np.float32)
            reproject(
                source=slope_img[0],
                destination=slope_on_dhdt,
                src_transform=slope_tx,
                src_crs=slope_src.crs,
                dst_transform=dhdt_tx,
                dst_crs=dhdt_src.crs,
                resampling=Resampling.bilinear
            )

            #Stable mask on dh/dt grid: slope < threshold
            stable_mask = np.isfinite(slope_on_dhdt) & (slope_on_dhdt < slope_thresh_deg)

            # Apply mask to dh/dt and drop NaNs
            dhdt_vals = dhdt_arr[stable_mask]
            dhdt_vals = dhdt_vals[np.isfinite(dhdt_vals)]
            n_stable = dhdt_vals.size
            if n_stable < min_stable_px:
                print(f"landform {idx}: too few stable pixels ({n_stable})")
                continue

            #Stats on stable pixels
            med = float(np.nanmedian(dhdt_vals))
            nmad = float(1.4826 * np.nanmedian(np.abs(dhdt_vals - med)))   # report (robust)
            std_dev = float(np.nanstd(dhdt_vals))                           # base random for Rolstad
            bias = float(np.nanmean(dhdt_vals))                             # systematic (should be ~0)

            #Area A for Rolstad scaling (m²). Prefer attribute if present, else geometry area
            shape_area_attr = None
            for key in ("Shape_Area", "Shape_area", "shape_area", "Area", "AREA"):
                if key in row and pd.notnull(row[key]):
                    try:
                        shape_area_attr = float(row[key])
                    except Exception:
                        shape_area_attr = None
                    break
            A = shape_area_attr if shape_area_attr is not None else float(geom.area)
            if A <= 0:
                print(f"landform {idx}: non-positive area")
                continue

            #Rolstad scaling of STD
            piL2 = np.pi * (L_m ** 2)
            scale = float(np.sqrt(piL2 / A)) if A >= piL2 else 1.0
            sigma_random_scaled = std_dev * scale

            #RMSE using scaled STD + bias (final σΔh)
            rmse = float(np.sqrt(sigma_random_scaled**2 + bias**2))

            # (optional): median of cropped dh/dt subset
            median_elev_dhdt = float(np.nanmedian(dhdt_arr))

            #Save record
            records.append({
                "landform_idx": idx,
                "n_stable_px": int(n_stable),
                "NMAD": nmad,                           # report
                "STD": std_dev,                         # reference
                "bias_mean": bias,                      # systematic
                "scale_factor": scale,                  # Rolstad
                "scaled_sigma_random": sigma_random_scaled,
                "RMSE_sigma_dh": rmse,                  # final σΔh
                "area_m2": A,
                "area_km2": A / 1e6,
                "piL2_km2": piL2 / 1e6,
                "median_elev_dhdt": median_elev_dhdt
            })

            print(
                f"landform {idx}: , NMAD={nmad:.4f}, STD={std_dev:.4f}, "
                f" scale={scale:.4f}, rand_scaled={sigma_random_scaled:.4f}, "
                f"RMSE={rmse:.4f}"
            )

        except Exception as e:
            print(f"Error processing landform {idx}: {e}")
            continue



CRS of slope raster: EPSG:6341
CRS of dh/dt raster: EPSG:6341
landform 0: , NMAD=0.0188, STD=0.0253,  scale=1.0000, rand_scaled=0.0253, RMSE=0.0285
landform 1: , NMAD=0.0252, STD=0.0270,  scale=1.0000, rand_scaled=0.0270, RMSE=0.0343
landform 2: , NMAD=0.0208, STD=0.0243,  scale=1.0000, rand_scaled=0.0243, RMSE=0.0347
landform 3: , NMAD=0.0482, STD=0.0504,  scale=1.0000, rand_scaled=0.0504, RMSE=0.0664
landform 4: , NMAD=0.0386, STD=0.1003,  scale=1.0000, rand_scaled=0.1003, RMSE=0.1137
landform 5: , NMAD=0.0388, STD=0.0815,  scale=1.0000, rand_scaled=0.0815, RMSE=0.0959
landform 6: , NMAD=0.0377, STD=0.1060,  scale=1.0000, rand_scaled=0.1060, RMSE=0.1184
landform 7: , NMAD=0.0430, STD=0.0697,  scale=1.0000, rand_scaled=0.0697, RMSE=0.0710
landform 8: , NMAD=0.0794, STD=0.0879,  scale=1.0000, rand_scaled=0.0879, RMSE=0.0920
landform 9: , NMAD=0.0000, STD=0.0000,  scale=1.0000, rand_scaled=0.0000, RMSE=0.0000
landform 10: , NMAD=0.0916, STD=0.0867,  scale=1.0000, rand_scaled=0.0867, RMS

In [5]:
# =============================
# Results table
# =============================
df = pd.DataFrame.from_records(records).sort_values("landform_idx").reset_index(drop=True)
#print("\nSummary:")
print(df)

# Compute and print plain average RMSE across landforms
if not df.empty:
    avg_rmse = df["RMSE_sigma_dh"].mean()
    print(f"\nAverage RMSE across {len(df)} landforms: {avg_rmse:.4f}")
else:
    print("\nNo valid landforms processed, cannot compute average RMSE.")

     landform_idx  n_stable_px      NMAD       STD  bias_mean  scale_factor  \
0               0      1572627  0.018760  0.025265  -0.013181           1.0   
1               1      1384884  0.025168  0.026990  -0.021161           1.0   
2               2      1394472  0.020832  0.024339  -0.024801           1.0   
3               3      1815805  0.048187  0.050433  -0.043228           1.0   
4               4      1965315  0.038584  0.100314  -0.053446           1.0   
..            ...          ...       ...       ...        ...           ...   
99             99       836092  0.000000  0.023791   0.005855           1.0   
100           100      1331648  0.000000  0.007155   0.000539           1.0   
101           101      1372410  0.000000  0.000000   0.000000           1.0   
102           102      1856831  0.024552  0.027127  -0.032665           1.0   
103           103       999135  0.043135  0.080375  -0.024373           1.0   

     scaled_sigma_random  RMSE_sigma_dh       area_